# Demo for playing around with ML model calling defined tools

### Imports

In [66]:
from helpers.promptGenerators import *
from helpers.modelHelpers import queryModel, parseModelResponseJSON
from helpers.toolHelpers import runToolRequests
import importlib

In [67]:
from dotenv import load_dotenv
load_dotenv(".env", override=True)
load_dotenv(".env.local", override=True)

True

### Tools definition

In [68]:
import callableTools
importlib.reload(callableTools)
TOOL_MODULES = [callableTools]

### User Query

In [69]:
USER_QUERY = "Searh for images of horses and send them to Vukasin"

### Load tools that could be called and generate prompts/schema for them
* Loads module where tools are defined
* Finds all methods labeled as "ai_callable"
* Generates schema for each method (output, input params)
* Generates prompt ML can understand

In [70]:
tool_prompts = generatePromptForTools(TOOL_MODULES)
tool_prompts

['    {        "tool": "Clock",        "function": "TimeNow",        "description": "Returns the current date and time formatted according to the server\'s standard date format.",        "output": "string",        "input_paramaters": {}    }    ',
 '    {        "tool": "Devices",        "function": "displayMessage",        "description": "Used to display message on screen/display.User can see the message on display.Provide a message as param.Just performes an action.",                "input_paramaters": {"message": "string"}    }    ',
 '    {        "tool": "Devices",        "function": "setLampBrightness",        "description": "Sets the brightness of a lamp device.Brightness param range from 0 to 255, 255 being the brightest.Returns the brightness of the lamp, not that usefull data.",                "input_paramaters": {"level": "int"}    }    ',
 '    {        "tool": "Messaging",        "function": "getUsers",        "description": "Retrieves the list of users who are eligible to

### Stage 1: Entich context 
* figuring out what tools to use
* running the tools
* preparing the output be passed to the next request

In [71]:
# Generate prompt for enrichment

#first extract only tools that provide output
output_only_tools_prompt = json.dumps(list(filter(lambda t: '"output":' in t, tool_prompts)))
context_enrich_prompt = generateContextEnrichmentPrompt(output_only_tools_prompt, USER_QUERY)

context_enrich_prompt

'Role: Middleware Request Interpreter\n\nYou are a middleware agent between a human user and a server.\n\nYour responsibilities are to:\n- Interpret the user\'s request.\n- Select the most appropriate data-providing tools from the available list.\n- Construct a structured request that the server can understand.\n- Use tools only to gather data in order to enrich context.\n\nTHIS IS A DATA-ENRICHMENT PHASE ONLY.\n\n====================\nTOOL SELECTION (ABSOLUTE)\n====================\n\nYou MUST NOT perform actions.\nYou MUST NOT select tools that cause side effects.\nMessaging, device control, system modification, or user interaction tools are STRICTLY FORBIDDEN in this phase.\n\nExamples of FORBIDDEN tools in this phase include (but are not limited to):\n- Messaging.sendMessage\n- Devices.displayMessage\n- Devices.setLampBrightness\n- System.PrankUser\n- System.CommitSuicide\n\n====================\nSTRICT OUTPUT RULES (ABSOLUTE)\n====================\n- Output MUST be valid JSON only

In [72]:
# Request completion from model for that prompt
res = queryModel(context_enrich_prompt)
res

' [\n  {\n    "tool": "Search",\n    "function": "findImages",\n    "input_paramaters": {\n      "search": "horses"\n    }\n  },\n  {\n    "tool": "Messaging",\n    "function": "sendMessage",\n    "input_paramaters": {\n      "recipient": "Vukasin",\n      "message": "[[images]]"\n    }\n  }\n]'

In [73]:
# Clean up response and parse
tool_requests_stage1 = parseModelResponseJSON(res)
tool_requests_stage1

[{'tool': 'Search',
  'function': 'findImages',
  'input_paramaters': {'search': 'horses'}},
 {'tool': 'Messaging',
  'function': 'sendMessage',
  'input_paramaters': {'recipient': 'Vukasin', 'message': '[[images]]'}}]

In [63]:
# Run tools
tool_outputs_stage1 = runToolRequests(tool_requests_stage1, TOOL_MODULES)
tool_outputs_stage1

[{'tool': 'Weather',
  'function': 'CurrentWeather',
  'description': 'Provides the current weather conditions for your location, including details such as the temperature and wind speed, so you can get an accurate sense of what the weather is like right now and plan your day accordingly.',
  'output': {'temp': 9.2, 'wind_speed': 7.5}}]

### Stage 2: Run actions with enriched context
* figuring out what tools to use
* using the output of previous tools to gather information
* generating tool params
* running the tools

In [64]:
# Generate prompt for calling action tools
actions_prompt = generateActionsPrompt(json.dumps(tool_prompts), tool_outputs_stage1, USER_QUERY)
actions_prompt

'Role: Middleware Action Executor\n\nYou are the SECOND middleware agent in a two-step pipeline.\n\nThis step is responsible ONLY for executing actions.\nContext enrichment and data gathering have already been completed.\n\nYour responsibilities are to:\n- Interpret the initial user input.\n- Interpret the output of the first AI (data-enrichment phase).\n- Select the most appropriate ACTION-ONLY tool.\n- Construct a structured request that can be understood by a server.\n\n====================\nSTRICT OUTPUT RULES (ABSOLUTE)\n====================\n- Output MUST be valid JSON only.\n- Output MUST be either:\n  - A JSON array following the exact schema below, OR\n  - An empty JSON array [].\n- Do NOT include explanations, reasoning, comments, markdown, or extra text.\n- Do NOT include leading or trailing whitespace.\n- Do NOT wrap the output in code blocks.\n- Do NOT echo the prompt, instructions, user input, or AI output.\n- Any output that is not valid JSON is a failure.\n\n===========

In [65]:
# Request completion from model for that prompt
tool_requests_stage2 = parseModelResponseJSON(queryModel(actions_prompt))
tool_requests_stage2

JSONDecodeError: Expecting ',' delimiter: line 1 column 141 (char 140)

In [ ]:
# Run tools
tool_outputs_stage2 = runToolRequests(tool_requests_stage2, TOOL_MODULES)
tool_outputs_stage2

[{'tool': 'Search',
  'function': 'findImages',
  'description': 'Finds images based on a query:Performs an online search for images based on a given query string.Parameters:    search (str): The search term or phrase to look for images.Returns:    list[str]: A list of URLs pointing to images relevant to the search query.Notes:    - The function retrieves images from public sources on the internet.    - The number and quality of returned images may vary depending on the search term.',
  'output': ['https://upload.wikimedia.org/wikipedia/commons/d/de/Nokota_Horses_cropped.jpg',
   'https://cdn.prod.website-files.com/66979546a02b17230c82da5d/674fb26699948a3f065b6d28_Onaqui-2005_Kimmer.jpg',
   'https://www.nps.gov/asis/learn/nature/images/horsesrunningbeach_2.jpg?maxwidth=1300&autorotate=false',
   'https://www.amnh.org/var/ezflow_site/storage/images/media/amnh/images/explore/ology-images/zoology/all-about-horses/carousel-hero---all-about-horses/4439248-3-eng-US/carousel-hero---all-about

### Stage 3: Generating final output
* using the output of previous tools to generate human readable response

In [ ]:
# Generate prompt for final response
final_response_prompt = generateFinalResponsePrompt(output_only_tools_prompt, tool_outputs_stage1 + tool_outputs_stage2, USER_QUERY)
final_response_prompt

'Role: Helpfull Assistant\n\nYou are a helpfull agent between a human user and a server.\n\nServer generated responses for this user request: \nSearh for images of horses and send them to Vukasin\n\nThe server used these tools and gave you an outputs:\n[{"tool": "Search", "function": "findImages", "description": "Finds images based on a query:Performs an online search for images based on a given query string.Parameters:    search (str): The search term or phrase to look for images.Returns:    list[str]: A list of URLs pointing to images relevant to the search query.Notes:    - The function retrieves images from public sources on the internet.    - The number and quality of returned images may vary depending on the search term.", "output": ["https://upload.wikimedia.org/wikipedia/commons/d/de/Nokota_Horses_cropped.jpg", "https://cdn.prod.website-files.com/66979546a02b17230c82da5d/674fb26699948a3f065b6d28_Onaqui-2005_Kimmer.jpg", "https://www.nps.gov/asis/learn/nature/images/horsesrunnin

In [ ]:
# Request completion from model for that prompt
final_response = queryModel(final_response_prompt)
final_response

' Based on your request and the information provided by the server, I have identified a number of images of horses that can be shared with Vukasin. Here are some beautiful horse images that were found during the search:\n\n1. <https://upload.wikimedia.org/wikipedia/commons/d/de/Nokota_Horses_cropped.jpg>\n2. <https://cdn.prod.website-files.com/66979546a02b17230c82da5d/674fb26699948a3f065b6d28_Onaqui-2005_Kimmer.jpg>\n3. <https://www.nps.gov/asis/learn/nature/images/horsesrunningbeach_2.jpg?maxwidth=1300&autorotate=false>\n4. <https://www.amnh.org/var/ezflow_site/storage/images/media/amnh/images/explore/ology-images/zoology/all-about-horses/carousel-hero---all-about-horses/4439248-3-eng-US/carousel-hero---all-about-horses_facebookshare_1200.jpg>\n5. <https://vetmed.tamu.edu/news/wp-content/uploads/sites/9/2024/09/Horse-greying-1.png>\n6. <https://img.lb.wbmdstatic.com/vim/live/webmd/consumer_assets/site_images/article_thumbnails/SEEDs/1800x1200-the-shire-horse-seed.jpg?resize=750px:*&ou

In [ ]:
print(final_response)

 Based on your request and the information provided by the server, I have identified a number of images of horses that can be shared with Vukasin. Here are some beautiful horse images that were found during the search:

1. <https://upload.wikimedia.org/wikipedia/commons/d/de/Nokota_Horses_cropped.jpg>
2. <https://cdn.prod.website-files.com/66979546a02b17230c82da5d/674fb26699948a3f065b6d28_Onaqui-2005_Kimmer.jpg>
3. <https://www.nps.gov/asis/learn/nature/images/horsesrunningbeach_2.jpg?maxwidth=1300&autorotate=false>
4. <https://www.amnh.org/var/ezflow_site/storage/images/media/amnh/images/explore/ology-images/zoology/all-about-horses/carousel-hero---all-about-horses/4439248-3-eng-US/carousel-hero---all-about-horses_facebookshare_1200.jpg>
5. <https://vetmed.tamu.edu/news/wp-content/uploads/sites/9/2024/09/Horse-greying-1.png>
6. <https://img.lb.wbmdstatic.com/vim/live/webmd/consumer_assets/site_images/article_thumbnails/SEEDs/1800x1200-the-shire-horse-seed.jpg?resize=750px:*&output-qua